In [1]:
from Selector import *
import sys
import time
from view.HTML import show
import json
from indicators.MA import *
from simulation.Simulator import *
from indicators.Trend import *
sys.path.append("../data")
import KDataDao as kds
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *
from simulation.SequenceSimulatorS import *
from simulation.ParallelSimulator import *
from strategy.sequence.MAStrategyS import *
from strategy.sequence.RStrategy import *
from indicators.TrendArea import *
from strategy.parallel.StrategyP import *
from strategy.parallel.BuyA import *
from strategy.parallel.SellA import *
from strategy.parallel.SellB import *
from strategy.parallel.SellS import *
from strategy.component.ConditionBean import *

In [2]:
class KStrategy:
    """
    K线策略
    """
    def __init__(self,trader=None,listK=None):
        self.point = 0 
        self.trend = TrendArea()
        self.condition = None
        self.listK = None
        self.trader = None
    def init(self,trader,listK):
        self.trader = trader
        self.listK = listK
        self.trend = TrendArea()
    def decide(self,i):
        k = self.listK[i]
        if self.trader.isHold(k.code):
            if k.isDown():
                self.trader.sell(k.code, k.date, k.close, condition="k1")
        else:
            if k.isUp():
                self.trader.buy(k.code, k.date, k.close, condition="k1")

In [3]:
#测试策略
# bDate = "2008-06-05"
bDate = "2015-03-05"
eDate = "2016-11-07"
market = "sza"
begin = time.time()
print 'runing......'
trader = Trader()
buys = [BuyA()]
sells = [SellA("a"),SellB('a'),SellS("a")]
s1 = StrategyP(trader,buys,sells)
# s1 = MAStrategyS(30)
sim1 = ParallelSimulator(s1)
sim1.run(market,bDate,eDate)
print 'complete %s second' %  (time.time()-begin)

runing......
2015-03-06
2015-03-07
2015-03-10
2015-03-11
2015-03-12
2015-03-13
2015-03-14
2015-03-17
2015-03-18
2015-03-19
2015-03-20
2015-03-21
2015-03-24
2015-03-25
2015-03-26
2015-03-27
2015-03-28
2015-03-31
2015-04-01
2015-04-02
2015-04-03
2015-04-04
2015-04-08
s>>>>>300295,s1
s>>>>>300359,s1
s>>>>>300364,s1
2015-04-09
s>>>>>300251,s1
2015-04-10
2015-04-11
2015-04-14
s>>>>>002363,s1
s>>>>>300052,s1
s>>>>>300226,s1
2015-04-15
s>>>>>000048,s1
s>>>>>002649,s1
s>>>>>300044,s1
2015-04-16
2015-04-17
2015-04-18
s>>>>>002148,s1
s>>>>>002152,s1
s>>>>>002270,s1
s>>>>>002312,s1
s>>>>>300353,s1
2015-04-21
2015-04-22
2015-04-23
2015-04-24
2015-04-25
2015-04-28
s>>>>>002269,s1
s>>>>>300087,s1
s>>>>>300179,s1
s>>>>>300261,s1
s>>>>>300407,s1
s>>>>>603898,s1
2015-04-29
s>>>>>000656,s1
2015-04-30
s>>>>>002306,s1
s>>>>>002597,s1
2015-05-01
2015-05-05
s>>>>>000791,a1
s>>>>>000952,s1
s>>>>>002101,s1
s>>>>>002245,s1
s>>>>>600198,s1
2015-05-06
s>>>>>600052,s1
2015-05-07
s>>>>>002108,s1
s>>>>>300131,s1
s>

s>>>>>900928,s1
2016-01-08
2016-01-09
s>>>>>002761,s1
s>>>>>300184,s1
2016-01-12
s>>>>>000717,s1
s>>>>>002149,s1
s>>>>>002221,s1
s>>>>>300395,s1
s>>>>>601566,s1
2016-01-13
s>>>>>000620,s1
s>>>>>000812,s1
s>>>>>002238,s1
s>>>>>300126,s1
s>>>>>300480,s1
s>>>>>600119,s1
s>>>>>600190,s1
s>>>>>600551,s1
s>>>>>601007,s1
s>>>>>601010,s1
s>>>>>601117,s1
s>>>>>601168,s1
s>>>>>601368,s1
s>>>>>601800,s1
s>>>>>603025,s1
s>>>>>603883,s1
2016-01-14
s>>>>>600133,s1
2016-01-15
s>>>>>000898,a1
s>>>>>002206,s1
s>>>>>002314,s1
s>>>>>600688,s1
s>>>>>600882,s1
s>>>>>601005,s1
s>>>>>601600,s1
s>>>>>603066,s1
2016-01-16
s>>>>>002610,s1
s>>>>>601177,s1
2016-01-19
2016-01-20
s>>>>>002110,s1
s>>>>>002291,s1
2016-01-21
s>>>>>002128,s1
s>>>>>300042,s1
s>>>>>600010,s1
s>>>>>600027,s1
s>>>>>600050,s1
s>>>>>600282,s1
s>>>>>600958,s1
s>>>>>601158,s1
s>>>>>601211,s1
s>>>>>601258,s1
s>>>>>601992,s1
s>>>>>603169,s1
s>>>>>603799,s1
s>>>>>900911,s1
2016-01-22
2016-01-23
s>>>>>000903,s1
s>>>>>603885,s1
2016-01-26
s>>>>>000

In [ ]:
# begin = time.time()
# print 'runing......'
# s12 = KStrategy2()
# sim12 = SequenceSimulator(s12)
# sim12.run(market,bDate,eDate,threadCount=0)
# print 'complete %s second' %  (time.time()-begin)

In [ ]:
#均线策略
begin = time.time()
print 'runing......'
s2 = MAStrategyS(60)
sim2 = SequenceSimulatorS(s2)
sim2.run(market,bDate,eDate,threadCount=0)
print 'complete %s second' %  (time.time()-begin)

runing......
thread-main begin run


In [ ]:
#K线策略
begin = time.time()
print 'runing......'
ks = KStrategy()
simK = SequenceSimulatorS(ks)
simK.run(market,bDate,eDate,threadCount=0)
print 'complete %s second' %  (time.time()-begin)

In [ ]:
#择时收益
sim3 = Simulator(RStrategy())
sim3.runSingle('zs_000001',begin=bDate,end=eDate)

In [ ]:
listK = kd.get_k_data('zs_000001',bDate,eDate)
data,volume = ViewDataFactory.getCandlestick(listK)

In [ ]:
avgEarningsLine = sim1.trader.earningsLine

oddsLine,win,lose,odds = sim2.getOdds()
# avgEarningsLine2,maxCode2,maxEarningsLine2,minCode2,minEarningsLine2 = sim12.earningsLines()
maEarningsLine,maxCode,maxEarningsLine,minCode,minEarningsLine = sim2.earningsLines()
kEarningsLine,kMaxCode,kMaxEarningsLine,kMinCode,kMinEarningsLine = simK.earningsLines()

rEarningsLine = sim3.trader.earningsLine


params = {}
params["maxCode"] = maxCode
params["minCode"] = minCode
params["avgEarningsLine"] = json.dumps(avgEarningsLine)
params["avgEarningsLine2"] = {}#json.dumps(avgEarningsLine2)
params["maxEarningsLine"] = json.dumps(maxEarningsLine)
params["minEarningsLine"] = json.dumps(minEarningsLine)
params["maEarningsLine"] = json.dumps(maEarningsLine)
params["rEarningsLine"] = json.dumps(rEarningsLine)
params["kEarningsLine"] = json.dumps(kEarningsLine)
params["oddsLine"] = json.dumps(oddsLine)
params["win"] = json.dumps(win)
params["lose"] = json.dumps(lose)
params["odds"] = json.dumps(odds)
params["data"] = json.dumps(data)
params["volume"] = json.dumps(volume)
clear_output()
s = show("Strategy.html",params)
